In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load and view data
data = pd.read_csv('ppp data.csv')
print(f"Shape of the data is: {data.shape}")
data.head()

Shape of the data is: (417253, 10)


,LoanAmount,City,State,Zip,NAICSCode,BusinessType,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502,111110.0,Subchapter S Corporation,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041,111110.0,Corporation,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024,111110.0,Corporation,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478,111110.0,Limited Liability Company(LLC),4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007,111110.0,Corporation,4/28/2020,Allegiance Bank,TX-18,5


In [3]:
# investigate rows with null values
data[data.isnull().any(axis=1)]

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,DateApproved,Lender,CD,LoanRange
2863,37053,WALLIS,TX,77485,111998.0,NaN,5/3/2020,"Wells Fargo Bank, National Association",TX-10,2
5759,21396,FLORESVILLE,TX,78114,112990.0,NaN,5/3/2020,"Wells Fargo Bank, National Association",TX-28,1
16293,15102,ROSENBERG,TX,77471,236115.0,NaN,5/3/2020,"Wells Fargo Bank, National Association",TX-22,1
28793,45001,DALLAS,TX,75228,238160.0,NaN,5/1/2020,"JPMorgan Chase Bank, National Association",TX-05,2
30297,136475,KENNEDALE,TX,76060,238210.0,NaN,5/1/2020,"JPMorgan Chase Bank, National Association",TX-06,6
...,...,...,...,...,...,...,...,...,...,...
416999,"e $150,000-350,000",Berkeley,TX,75081,NaN,Limited Liability Company(LLC),5/1/2020,"JPMorgan Chase Bank, National Association",TX-32,7
417101,"e $150,000-350,000",RIVERSIDE,TX,75247,NaN,Corporation,5/1/2020,"JPMorgan Chase Bank, National Association",TX-30,7
417135,"e $150,000-350,000",REEDLEY,TX,78753,NaN,Corporation,5/1/2020,"JPMorgan Chase Bank, National Association",TX-10,7
417154,"e $150,000-350,000",Yucaipa,TX,77598,NaN,Corporation,5/1/2020,"JPMorgan Chase Bank, National Association",TX-22,7


In [4]:
# only take rows of businesses that CD is not null

data = data[data['CD'].notna()]
data.head()

#drop rows with NA cities
data = data[data['City'].notna()]

# drop rows with NA NAICS code
data = data[data['NAICSCode'].notna()]

#drop rows with NA business type
data = data[data['BusinessType'].notna()]

data.isnull().sum()

data.shape

data.head()

data['NAICSCode'] = data['NAICSCode'].astype(int)

data

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502,111110,Subchapter S Corporation,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041,111110,Corporation,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024,111110,Corporation,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478,111110,Limited Liability Company(LLC),4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007,111110,Corporation,4/28/2020,Allegiance Bank,TX-18,5
...,...,...,...,...,...,...,...,...,...,...
417248,"e $150,000-350,000",Lafayette,TX,77064,541611,Limited Liability Company(LLC),5/1/2020,"JPMorgan Chase Bank, National Association",TX-07,7
417249,"e $150,000-350,000",SANTA ROSA,TX,78205,531312,Corporation,4/14/2020,Vantage Bank Texas,TX-35,7
417250,"e $150,000-350,000",RANCHO CORDOVA,TX,77803,442210,Limited Liability Company(LLC),4/27/2020,"Spirit of Texas Bank, SSB",TX-17,7
417251,"e $150,000-350,000",LOS ALTOS,TX,78230,492210,Corporation,4/30/2020,Randolph-Brooks FCU,TX-21,7


In [5]:
## read in data for NAICS code

naics = pd.read_csv('US-Business-Profiles-By-Sales-and-Employees (2).csv')

naics.head()

,NAICS 1 Code,NAICS 1 Description
0,111110,Soybean Farming
1,111120,Oilseed (except Soybean) Farming
2,111130,Dry Pea and Bean Farming
3,111140,Wheat Farming
4,111150,Corn Farming


In [6]:
naics.dtypes
naics[naics.isnull().any(axis=1)]


naics['NAICS 1 Code'] = naics['NAICS 1 Code'].astype(int)

naics

,NAICS 1 Code,NAICS 1 Description
0,111110,Soybean Farming
1,111120,Oilseed (except Soybean) Farming
2,111130,Dry Pea and Bean Farming
3,111140,Wheat Farming
4,111150,Corn Farming
...,...,...
1040,926140,Regulation of Agricultural Marketing and Commo...
1041,926150,"Regulation, Licensing, and Inspection of Misce..."
1042,927110,Space Research and Technology
1043,928110,National Security


In [28]:
## merge tables 

loan_data = pd.merge(data, naics, left_on='NAICSCode', right_on='NAICS 1 Code')


# drop categorical columns
loan_data_top = loan_data.drop(columns=['City', 'DateApproved', 'CD', 'LoanAmount', 'BusinessType', 'State', 'NAICS 1 Description'], axis=1)

# loan_data = pd.get_dummies(data, columns=["BusinessType"])

loan_data_top.head()


,Zip,NAICSCode,Lender,LoanRange,NAICS 1 Code
0,77502,111110,"Capital One, National Association",6,111110
1,77041,111110,Comerica Bank,6,111110
2,75024,111110,Veritex Community Bank,5,111110
3,77478,111110,Allegiance Bank,5,111110
4,77007,111110,Allegiance Bank,5,111110


In [29]:
# drop lenders with loans of 5 or more
counts = loan_data_top['Lender'].value_counts()

loan_data_top = loan_data_top.loc[loan_data_top['Lender'].isin(counts.index[counts > 5])]
loan_data_top.shape

loan_data_top.head()


,Zip,NAICSCode,Lender,LoanRange,NAICS 1 Code
0,77502,111110,"Capital One, National Association",6,111110
1,77041,111110,Comerica Bank,6,111110
2,75024,111110,Veritex Community Bank,5,111110
3,77478,111110,Allegiance Bank,5,111110
4,77007,111110,Allegiance Bank,5,111110


In [30]:
# Assign X (data) and y(target)
X = loan_data_top.drop("Lender", axis=1)
y = loan_data_top["Lender"]
print(X.shape, y.shape)

(396227, 4) (396227,)


In [31]:
# Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [32]:
# Create a logistic Regression Model
classifier = LogisticRegression()
classifier

LogisticRegression()

In [33]:
# Fit model using the training data
classifier.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# evaluate the model's performance using the test data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
# Make predictions
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

In [ ]:
# View predictions along with their probabilities of being within each class 
##  this will be the difficult part of our project because data is not binary like example: male/female
probs = classifier.predict_proba(X_test)
pred_df = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_test, 
    "P(Female)": [p[0] for p in probs], 
    "P(Male)": [p[1] for p in probs]
    }).reset_index(drop=True)

pred_df.head(10)

In [ ]:
# Model Evaluation
# Create a Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predictions)

In [ ]:
# Label the confusion Matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Neg: {tn}   False Pos: {fp}")
print(f"False Neg: {fn}  True Pos: {tp}")

In [ ]:
# Plot confusion matrix
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(classifier, X_test, y_test, cmap="Blues")

In [ ]:
# Precision and Recall
### we don't technically have a "positive" label, like we would if we were classifying emails as Spam, or patients as having cancer. 
## This means that we need to tell our precision and recall functions which class label we want to consider to be our "positive" class i.e. which lender
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
## Random tests of lenders first test is JPMorgan Chase Bank, National Association, the lender with the most loans
precision_score(y_test, predictions, pos_label="JPMorgan Chase Bank, National Association")

In [ ]:
## Random tests of lenders second test is First National Bank of Louisiana, random test of lender with lower loan count of 10 loans
precision_score(y_test, predictions, pos_label="First National Bank of Louisiana")

In [ ]:
#  what is the harmonic mean of JP Morgan from first test
f1_score(y_test, predictions, pos_label="JPMorgan Chase Bank, National Association")

In [ ]:
#  what is the harmonic mean of First National Bank of Louisiana from second test
f1_score(y_test, predictions, pos_label="First National Bank of Louisiana")